# Data Preparation

### Imports

In [6]:
import investpy
import yfinance as yf
import pandas as pd
import requests 

## US companies ticket extraction

In [ ]:
us_stocks = investpy.stocks.get_stocks(country='united states')
us_stocks.to_csv('companies/us_stocks.csv', index=False)
print(us_stocks.head())

## Stocks price extraction from yahoo finance

In [ ]:
tickers = us_stocks['symbol'].tolist()

for company in us_stocks.iterrows():
    ticker = company[1]["symbol"]
    try:
        ticker_obj = yf.Ticker(ticker)
        hist = ticker_obj.history(period="max")
        hist.to_csv(f"companies\\price\\{company[1]["name"]}-{ticker}.csv", index=False)
        print(ticker)
    except Exception as e:
        print(e)

## Debt ratio extraction from FMP

In [ ]:
URL = "https://financialmodelingprep.com/api/v3/ratios/AAPL"

params = {
    'limit': '200',
    'apikey': '*'
}

for company in us_stocks.iterrows():
    ticker = company[1]["symbol"]
    
    try:
        response = requests.get(URL, params=params)
        json_res = response.json()
        company_debtR_history = {"Date": [], "DebtRatio": []}
        for year_data in json_res:
            if year_data['debtRatio'] != 0:
                company_debtR_history["Date"].append(year_data['date'])
                company_debtR_history["DebtRatio"].append(year_data['debtRatio'])
        
        company_df = pd.DataFrame(company_debtR_history)
        company_df.to_csv(f"companies/debt/{company[1]["name"]}-{ticker}.csv", index=False)
        print(ticker)
    except Exception as e:
        print(e)

## Concatenate close price of a stock with debt ratio

In [31]:
for company in us_stocks.iterrows():
    ticker = company[1]["symbol"]
    
    try:
        stocks = pd.read_csv(f"companies/price/{company[1]["name"]}-{ticker}.csv")
        debt = pd.read_csv(f"companies/debt/{company[1]["name"]}-{ticker}.csv")
        
        stocks['Date'] = pd.to_datetime(stocks['Date'], utc=True).dt.date
        debt['Date'] = pd.to_datetime(debt['Date']).dt.date
        
        combined_data = pd.merge(stocks, debt, on='Date', how='inner')
        combined_data = combined_data.dropna()
        
        only_close_debt = combined_data[['Date', 'Close', 'DebtRatio']]
        
        print(ticker)
        if not only_close_debt.empty:
            print(ticker)
            only_close_debt.to_csv(f"companies/combined/{company[1]["name"]}-{ticker}.csv", index=False)
    except Exception as e:
        print(e)

BA
BA
GM
CVX
CVX
C
C
BAC
BAC
T
T
CAT
CAT
INTC
INTC
MSFT
MSFT
AA
AA
F
F
EBAY
EBAY
DELL
DIS
DIS
DOW
CSCO
CSCO
DE
DE
FDX
FDX
GIS
GIS
GLW
GLW
GT
GT
GS
JPM
JPM
KMB
KMB
KHC
KO
KO
MCD
MCD
LLY
LLY
ORCL
ORCL
MRK
MRK
MSI
MSI
MMM
MMM
ODP
ODP
CAKFF
PMMAF
VRTX
VRTX
MNST
MNST
TEVA
TEVA
FITB
FITB
SHLDQ
CTAS
CTAS
ADSK
ADSK
DISH
GILD
GILD
LIFE
GOOGL
FISV
LBTYA
PDLI
ADBE
ADBE
BB
QCOM
QCOM
SINA
BIDU
SOHU
DISCA
RBBN
AMAT
AMAT
STLD
STLD
CDNS
CDNS
BBBY
MCHP
MCHP
WYNN
ISRG
NDAQ
RYAAY
RYAAY
APLO
INFY
JBLU
SPAR
HSIC
HSIC
PAYX
PAYX
VRSN
VRSN
AAPL
AAPL
FAST
FAST
XRAY
XRAY
CROX
ZION
ZION
NTRS
NTRS
CTXS
LAMR
LAMR
PTEN
PTEN
ATLC
CHRW
CHRW
SIRI
SIRI
ALPN
PCAR
PCAR
PAAS
PAAS
AMZN
AMZN
ROST
ROST
VIAV
VIAV
NTES
NTAP
NTAP
ETFC
GRMN
COST
COST
CHKP
CHKP
LRCX
LRCX
CIEN
CIEN
[Errno 2] No such file or directory: 'companies/price/IAC/InterActiveCorp-IAC.csv'
INTU
INTU
EXPE
TCOM
UTSI
CTSH
CTSH
RMBS
RMBS
NIHD
YRCW
AKAM
KLAC
KLAC
JNPR
ATVI
FLEX
FLEX
AMGN
AMGN
XLNX
EXPD
EXPD
EA
EA
PDCO
PDCO
TROW
TROW
BIIB
BIIB
QRTEA
SCHW
SCHW
HBA

## 